# A61H CPC Classification - A61H1100

<font size="3">

#### About the Module:
<I><span style="font-family:Arial">This module work to identify patents in A61H1100 class. Class detail is as follow<br></span>
<span style="font-family:Arial">A61H11/00	•	belts, strips or comb for massages<br></span>
<span style="font-family:Arial">A61H2011/005 •	with belt or strap expanding and contracting around an encircled body part<br></span>
<span style="font-family:Arial">A61H11/02	•	Massage devices with strips oscillating lengthwise<br></span>


#### Input needed: 
<I><span style="font-family:Arial">This module use libraries, function and processed text data present in initial_processing module</span></I>

#### Output expected: 
<I><span style="font-family:Arial">This module will identify patents in A61H1100 class in test data.<br>This output is pass to the module final_file. No output is expected in this module alone.</span></I>

#### Related modules: 
<I><span style="font-family:Arial">This module is calling initial_processing module and is called by final_file module</span></I>

#### Who and when: 
<I><span style="font-family:Arial">Last Modified by : Nishant Chauhan</span><br>
<span style="font-family:Arial">Last Modified on : 23-July-2020</span><br>
<span style="font-family:Arial">Version no       : 2</span><br>
<span style="font-family:Arial">Developed by     : Nishant Chauhan </span><br></font></I>


In [1]:
import os
import sys
import import_ipynb
from Initial_processing import *

importing Jupyter notebook from Initial_processing.ipynb
Loading of module Initial processing is successfully complete.


In [2]:
df['A61H1100'] = df['CPC'].str.contains('A61H11|A61H0011|A61H 11').astype('int')
df['A61H1100'].value_counts()

0    3803
1     114
Name: A61H1100, dtype: int64

## Model training and prediction

In [3]:
y_train = df['A61H1100']

model_title , score_title = train_model(X_title_dtm, y_train)

y_pred_class_title = model_title.predict(X_title_test_dtm)

In [4]:
model_abstract , score_abstract = train_model(X_abstract_dtm, y_train)

y_pred_class_abstract = model_abstract.predict(X_abstract_test_dtm)

In [5]:
model_tab , score_tab = train_model(X_tab_dtm, y_train)

y_pred_class_tab = model_tab.predict(X_tab_test_dtm)

print("Training data accuracy score : " ,((score_abstract.mean() + score_title.mean() + score_tab.mean())/3))

Training data accuracy score :  0.9722572820565841


In [6]:
df_first = pd.DataFrame(df_test[['number','title','abstract']])

In [7]:
df_first['title_prediction'] = y_pred_class_title
df_first['abstract_prediction'] = y_pred_class_abstract
df_first['all_text'] = X_all

## Predicting using words 
#### Need improvement, low number of patents, no specific keywords)

In [8]:
# AND Operator keywords

A61H1100 = []

In [15]:
# NEAR operator keywords (List1 NEAR5 List2)

A61H1100_list1 = ['belt', 'strip', 'comb', 'strap']
A61H1100_list2 = ['massag']

In [16]:
#SSTO command e.g. search "eye massage device" as single word

A61H1100_word = []

In [17]:
new_col = []
for i in range(len(df_first['all_text'])):
    check_word = (near_operator(df_first['all_text'][i],A61H1100_list1,A61H1100_list2, near=4) or 
                 and_operator(df_first['all_text'][i],A61H1100) or
                 ssto(df_first['all_text'][i], A61H1100_word)) 
    new_col.append(check_word)
    
df_first['word_predict'] = new_col

In [18]:
df_first['A61H1100'] = df_first['word_predict'] + df_first['title_prediction'] + df_first['abstract_prediction']

for row in range(len(df_first['A61H1100'])):
    if df_first.loc[row, 'A61H1100'] > 0:
        df_first.loc[row, 'A61H1100'] = 1
    else:
        df_first.loc[row, 'A61H1100'] = 0

In [19]:
df_second = df_first.drop(['title_prediction','abstract_prediction', 'word_predict'],axis=1)

## Sub-classification of A61H11/00

### A61H2011/005 • with belt or strap expanding and contracting around an encircled body part

In [21]:
# AND Operator keywords
A61H2011005 = ['belt encircl body part', 'strap encircl body part']

# NEAR operator keywords (List1 NEAR5 List2)
A61H2011005_list1 = ['belt', 'strap', 'strip']
A61H2011005_list2 = ['legs', 'hips', 'thigh', 'arms', 'limb', 'shoulder', 'hand', 'abdomen']

#SSTO command e.g. search "eye massage device" as single word
A61H2011005_word = []


In [22]:
new_col_2011005 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H1100'][i] == 1:
        check_word_2011005 = int((near_operator(df_second['all_text'][i],A61H2011005_list1,A61H2011005_list2, near=4)or 
                           and_operator(df_second['all_text'][i],A61H2011005) or
                           ssto(df_second['all_text'][i], A61H2011005_word)))
    else:
        check_word_2011005 = 0
    
    new_col_2011005.append(check_word_2011005)
    
df_second['A61H2011005'] = new_col_2011005

### A61H11/02 • Massage devices with strips oscillating lengthwise

In [24]:
# AND Operator keywords

A61H1102 = ['strip oscillat massag', 'strap oscillat massag','belt oscillat massag'
           'strip vibrat massag', 'strap vibrat massag','belt vibrat massag']

# NEAR operator keywords (List1 NEAR5 List2)

A61H1102_list1 = ['belt', 'strap', 'strip']
A61H1102_list2 = ['oscilla', 'vibrat']

#SSTO command e.g. search "eye massage device" as single word

A61H1102_word = []

In [25]:
new_col_1102 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H1100'][i] == 1:
        check_word_1102 = int((near_operator(df_second['all_text'][i],A61H1102_list1,A61H1102_list2, near=4)or 
                           and_operator(df_second['all_text'][i],A61H1102) or
                           ssto(df_second['all_text'][i], A61H1102_word)))
    else:
        check_word_1102 = 0
    
    new_col_1102.append(check_word_1102)
    
df_second['A61H1102'] = new_col_1102

## Final step

In [28]:
for i in range(4,len(df_second.columns)):
    df_second.iloc[:,i].replace(1,df_second.columns[i], inplace=True)
    df_second.iloc[:,i].replace(0,'', inplace=True)

In [29]:
cols = ['A61H1100', 'A61H2011005', 'A61H1102']

df_second['prediction'] = ''
for i in range(len(df_second)):
    for col in cols:
        if not df_second.loc[i,col] == '':
            df_second.loc[i,'prediction'] = (df_second.loc[i,'prediction'] + ' | '
                                                        + df_second.loc[i,col])

df_second['prediction'] = df_second['prediction'].str[3:]

In [30]:
df_final = df_second.drop(
    columns= ['A61H1100', 'A61H2011005', 'A61H1102', 'all_text'], axis=1)

In [31]:
from datetime import datetime
dt_string = datetime.now().strftime("%d/%b/%Y - %H:%M %p")
print("Module A61H1100 is successfully loaded on",dt_string)

Loading of module A61H0500 is successfully complete.
